### AVISO: ESTE JUPYTER NOTEBOOK VIENE DE UNA VERSION DE DATABRICKS NOTEBOOK. NO LO HE TESTEADO AQUI COMPLETAMENTE Y LA PARTE EN LA QUE SE APLICA RANDOM FORESTS PUEDE DAR ALGUN ERROR(AUNQUE NO CREO). LOS RESULTADOS SOBRE LOS QUE HABLO ABAJO SON LOS QUE ME HAN DADO A MI AL APLICAR EL ALGORITMO, Y AL VOLVERLO A EJECUTAR PUEDEN VARIAR UN POCO (PERO NO VARIARA SIGNIFICATIVAMENTE). EN EL PDF Y VERSION DE DATABRICKS QUE ACOMPAÑAN A ESTE NOTEBOOK SI VIENEN LOS RESULTADOS FINALES AL EJECUTAR EL ALGORITMO.

###### UTILIZAMOS RANDOM FORESTS PARA ANALIZAR LA RELACION ENTRE LAS DISTINTAS VARIABLES, ASI COMO LA INFLUENCIA QUE TIENEN LA HORA DEL DIA O EL DIA DE LA SEMANA. ESTE ALGORITMO NOS VA A DAR PARA CADA VARIABLE QUE ANALICEMOS (NO2,CO2,PART,03 Y SO2) CUALES SON LAS VARIABLES QUE MEJOR "EXPLICAN" A LA ANALIZADA. DE ESTA FORMA PODEMOS VER DE MANERA RAPIDA SI A GRAN ESCALA LOS DATOS TIENEN SENTIDO Y POR LO TANTO SE PUEDEN CONSIDERAR LAS MEDICIONES COMO FIABLES, O SI PRESENTAN ALGUNAS INCONGRUENCIAS. AQUI ANALIZAMOS SOLO LOS DATOS DEL PALACIO DE CONGRESOS PARA TODOS LOS DATOS QUE HAY DESDE 2009 HASTA MARZO DE 2017

In [1]:
import pandas as pd
import datetime
import numpy as np

import matplotlib.pyplot as plt
import matplotlib

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics  import roc_auc_score

In [11]:
congr_datasetDF  = pd.DataFrame.from_csv('https://raw.githubusercontent.com/oslugr/contaminAND/master/datos/contaminAND-gr-congresos.csv').reset_index()
congr_datasetDF['date'] = congr_datasetDF['date'].apply(lambda x: pd.to_datetime(x,dayfirst = True))
congr_datasetDF

,date,NO2,CO,PART,O3,SO2
0,2009-07-20 00:10:00,28,1708,50,74,6
1,2009-07-20 00:20:00,28,1694,51,76,6
2,2009-07-20 00:30:00,27,1700,45,77,6
3,2009-07-20 00:40:00,25,1662,33,79,6
4,2009-07-20 00:50:00,26,1680,23,76,6
5,2009-07-20 01:00:00,25,1673,23,78,6
6,2009-07-20 01:10:00,28,1692,29,66,6
7,2009-07-20 01:20:00,34,1729,31,48,6
8,2009-07-20 01:30:00,35,1723,35,53,6
9,2009-07-20 01:40:00,33,1717,42,58,6


In [19]:
#OBTENEMOS DOS NUEVAS COLUMNAS CON EL DIA DE LA SEMANA Y LA HORA DEL DIA(EN MINUTOS)
congr_datasetDF['WEEKDAY'] = congr_datasetDF['date'].apply(lambda x: x.weekday())
congr_datasetDF['TIME_minutesofday'] = congr_datasetDF['date'].apply(lambda x: x.hour*60+x.minute)
congr_datasetDF = congr_datasetDF.drop(['date'],axis=1)

In [20]:
congr_datasetDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388892 entries, 0 to 388891
Data columns (total 7 columns):
NO2                  388892 non-null object
CO                   388892 non-null object
PART                 388892 non-null object
O3                   388892 non-null object
SO2                  388892 non-null object
WEEKDAY              388892 non-null int64
TIME_minutesofday    388892 non-null int64
dtypes: int64(2), object(5)
memory usage: 20.8+ MB


In [21]:
congr_datasetDF = congr_datasetDF.apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [22]:
congr_datasetDF.isnull().any()

NO2                   True
CO                    True
PART                  True
O3                    True
SO2                   True
WEEKDAY              False
TIME_minutesofday    False
dtype: bool

In [23]:
congr_datasetDF.isnull().sum()

NO2                   7156
CO                    5280
PART                 14772
O3                    2824
SO2                   4103
WEEKDAY                  0
TIME_minutesofday        0
dtype: int64

### FORZAMOS VALORES NUMERICOS. ALLI DONDE NO ES POSIBLE SERA PORQUE NO HABIA DATOS, O ERAN TEXTO. ESOS PASAN A SER NP.NAN, AHORA MAS ABAJO LES METEMOS EL VALOR MEDIO DE LA COLUMNA A LA QUE PERTENEZCAN. AUNQUE SE PUEDE VER ARRIBA QUE EL UNICO QUE LE AFECTA DE VERDAD ES A LA COLUMNA 'PART' QUE HA PERDIDO CASI 15000 VALORES, EL RESTO ESTAN BASTANTE BIEN

## A continuacion hacemos lo siguiente:
        Limpiar el dataset y dejarlo preparado para que se pueda aplicar Random Forests.
        Añadir columnas relativas al dia de la semana, hora del dia y si es fin de semana o no (considerando V-D y solo S-D)

In [24]:
def media(column):
    column = column.fillna(value=column.mean())
    return column
congr_datasetDF = congr_datasetDF.apply(media,axis=0)

In [25]:
#VARIABLE DUMMY EN FUCION DE SI ES FIN DE SEMANA O NO(CONSIDERADO COMO V-D)
def fin_semana(row):
    if ((row.WEEKDAY == 0) or (row.WEEKDAY >= 5 )):
        return 1
    else:
        return 0
congr_datasetDF['WEEKEND_VD'] = congr_datasetDF.apply(fin_semana,axis=1)

def fin_semana(row):
    if ((row.WEEKDAY == 0) or (row.WEEKDAY == 6)):
        return 1
    else:
        return 0

#VARIABLE DUMMY EN FUCION DE SI ES FIN DE SEMANA O NO(CONSIDERADO COMO S-D)
congr_datasetDF['WEEKEND_SD'] = congr_datasetDF.apply(fin_semana,axis=1)

In [26]:
congr_datasetDF

,NO2,CO,PART,O3,SO2,WEEKDAY,TIME_minutesofday,WEEKEND_VD,WEEKEND_SD
0,28.0,1708.0,50.0,74.0,6.0,0,10,1,1
1,28.0,1694.0,51.0,76.0,6.0,0,20,1,1
2,27.0,1700.0,45.0,77.0,6.0,0,30,1,1
3,25.0,1662.0,33.0,79.0,6.0,0,40,1,1
4,26.0,1680.0,23.0,76.0,6.0,0,50,1,1
5,25.0,1673.0,23.0,78.0,6.0,0,60,1,1
6,28.0,1692.0,29.0,66.0,6.0,0,70,1,1
7,34.0,1729.0,31.0,48.0,6.0,0,80,1,1
8,35.0,1723.0,35.0,53.0,6.0,0,90,1,1
9,33.0,1717.0,42.0,58.0,6.0,0,100,1,1


### ANALIZAMOS AHORA QUE VARIABLES INFLUYEN MAS EN LOS NIVELES DE NO2

In [25]:
X_bis = congr_datasetDF.copy()
y = X_bis.pop('NO2')
    
model = RandomForestRegressor(n_estimators = 100,oob_score=True,n_jobs=-1)
model.fit(X_bis,y)
score = model.oob_score_
 
feature_importances = model.feature_importances_
feature_importances_Series = pd.Series(data=feature_importances,index=X_bis.columns).sort_values(ascending=False)
features = feature_importances_Series



In [ ]:
print 'Precision del modelo para el analisis de la variable "NO2": ',score

In [27]:
print 'Variables que explican los valores de NO2 \n \n',features

### ANALIZAMOS AHORA QUE VARIABLES INFLUYEN MAS EN LOS NIVELES DE PARTICULAS

In [31]:
X_bis = congr_datasetDF.copy()
y = X_bis.pop('PART')
    
model = RandomForestRegressor(n_estimators = 100,oob_score=True,n_jobs=-1)
model.fit(X_bis,y)
score = model.oob_score_
 
feature_importances = model.feature_importances_
feature_importances_Series = pd.Series(data=feature_importances,index=X_bis.columns).sort_values(ascending=False)
features = feature_importances_Series


In [32]:
print 'Precision del modelo para el analisis de la variable "PART": ' ,score

In [33]:
print 'Variables que explican los valores de PART \n \n',features

### ANALIZAMOS AHORA QUE VARIABLES INFLUYEN MAS EN LOS NIVELES DE S02

In [37]:
X_bis = congr_datasetDF.copy()
y = X_bis.pop('SO2')
    
model = RandomForestRegressor(n_estimators = 100,oob_score=True,n_jobs=-1)
model.fit(X_bis,y)
score = model.oob_score_
 
feature_importances = model.feature_importances_
feature_importances_Series = pd.Series(data=feature_importances,index=X_bis.columns).sort_values(ascending=False)
features = feature_importances_Series


In [38]:
print('Precision del modelo para el analisis de la variable "S02": ',score)

In [39]:
print 'Variables que explican los valores de S02 \n \n',features

### ANALIZAMOS AHORA QUE VARIABLES INFLUYEN MAS EN LOS NIVELES DE O3

In [43]:
X_bis = congr_datasetDF.copy()
y = X_bis.pop('O3')
    
model = RandomForestRegressor(n_estimators = 100,oob_score=True,n_jobs=-1)
model.fit(X_bis,y)
score = model.oob_score_
 
feature_importances = model.feature_importances_
feature_importances_Series = pd.Series(data=feature_importances,index=X_bis.columns).sort_values(ascending=False)
features = feature_importances_Series


In [44]:
print 'Precision del modelo para el analisis de la variable "O3": ',score

In [45]:
print 'Variables que explican los valores de 03 \n \n',features

### CONCLUSIONES INICIALES: